In [1]:
import numpy as np

import import_ipynb
from grid_world import ACTION_SPACE, standard_grid
from iterative_policy_evaluation_deterministic import print_values, print_policy

importing Jupyter notebook from grid_world.ipynb
importing Jupyter notebook from iterative_policy_evaluation_deterministic.ipynb


In [35]:
SMALL_ENOUGH = 1e-3
GAMMA = 0.9

In [51]:
def get_transition_probs_and_rewards(grid) :
    #key is (s, a, s')
    #transition_probs[(s, a, s')] = p(s' | s, a)
    #any key not present will be considered to have a prob of 0
    transition_probs = {}
    
    #use deterministic rewards
    #rewards[(s, a, s')] or rewards[s']
    rewards = {}
    for i in range(grid.rows) :
        for j in range(grid.cols) :
            s = (i, j)
            if not grid.is_terminal(s) :
                for a in ACTION_SPACE :
                    s2 = grid.get_next_state(s, a)
                    transition_probs[(s, a, s2)] = 1
                    if s2 in grid.rewards:
                        rewards[(s, a, s2)] = grid.rewards[s2]  
    return transition_probs, rewards

def evaluate_deterministic_policy(grid, policy) :
    #initialize V(s) = 0
    V = {}
    for s in grid.all_states() :
        V[s] = 0
        
    gamma = 0.9 #discount factor
    it = 0
    #repeat till convergance
    while True:
        biggest_change = 0
        for s in grid.actions :
            V_old = V[s]
            V_new = 0 #this will accumulate the answer
            for a in ACTION_SPACE :
                for s2 in grid.all_states() : 
                    #action probability
                    action_prob = 1 if policy.get(s) == a else 0
                    
                    #reward
                    r = rewards.get((s, a, s2), 0)
                    
                    V_new += action_prob * transition_probs.get((s, a, s2), 0) * (r + gamma * V[s2])
            V[s] = V_new
            biggest_change = max(biggest_change, np.abs(V_old - V_new))
        #print(f'Iter {it}, biggest change {biggest_change}')
        #print_values(V, grid)
        it += 1
        if biggest_change < SMALL_ENOUGH :
            break
    return V

In [59]:
if __name__ == '__main__' :
    grid = standard_grid()
    
    transition_probs, rewards = get_transition_probs_and_rewards(grid)
    
    print('Rewards')
    print_values(grid.rewards, grid)
    
    policy = {}
    for s in grid.actions.keys() :
        policy[s] = np.random.choice(ACTION_SPACE)
    
    #initial random policy
    print('Initial Policy')
    print_policy(policy, grid)
    
    while True :
        delta = 0
        #policy evaluation
        V = evaluate_deterministic_policy(grid, policy)
            
        #policy improvement step
        is_policy_stable = True
        for s in grid.actions.keys() : #all_non_terminal_states
            A_old = policy[s]
            A_new_prob_over_a = [0 for i in ACTION_SPACE]
            for count_a, a in enumerate(ACTION_SPACE) :
                for s2 in grid.all_states() :
                    A_new_prob_over_a[count_a] += transition_probs.get((s, a, s2), 0) * (rewards.get((s, a, s2), 0) + GAMMA * V[s2])
            A_new = np.argmax(A_new_prob_over_a)
            policy[s] = ACTION_SPACE[A_new]
            if A_old != policy[s] : 
                is_policy_stable = False
        
        if is_policy_stable :
            break
            
    #final values
    print('Final values')
    print_values(V, grid)
    #final policy
    print('Final Policy')
    print_policy(policy, grid)

Rewards
+---+---+---+----+
| 0 | 0 | 0 |  1 |
+---+---+---+----+
| 0 | 0 | 0 | -1 |
+---+---+---+----+
| 0 | 0 | 0 |  0 |
+---+---+---+----+
Initial Policy
+---+---+---+---+
| L | U | D |   |
+---+---+---+---+
| D |   | L |   |
+---+---+---+---+
| L | D | R | R |
+---+---+---+---+
Final values
+-------+-------+-------+-------+
| 0.810 | 0.900 | 1.000 | 0.000 |
+-------+-------+-------+-------+
| 0.729 | 0.000 | 0.900 | 0.000 |
+-------+-------+-------+-------+
| 0.656 | 0.729 | 0.810 | 0.729 |
+-------+-------+-------+-------+
Final Policy
+---+---+---+---+
| R | R | R |   |
+---+---+---+---+
| U |   | U |   |
+---+---+---+---+
| U | R | U | L |
+---+---+---+---+
